In [7]:
import os
import random
import time
from typing import List, Tuple
from urllib.request import urlopen

import numpy as np
import scipy
import pandas as pd
import sklearn
import requests
import matplotlib.pyplot as plt

In [12]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor

In [13]:
!python --version

Python 3.8.5


In [21]:
"""
タスクをランダムに取得するサンプルプログラムです。
実行には python3 環境が必要です。
TOKEN 変数を書き換えて実行してください。
"""

import os
import random
import time
import json
from typing import Tuple
from urllib.request import urlopen

# ゲームサーバのアドレス
GAME_SERVER = os.getenv('GAME_SERVER', 'https://contest.2021-spring.gbc.tenka1.klab.jp')
# あなたのトークン
TOKEN = "b16c91e37d4c75777beba2eb755056fd"#os.getenv('TOKEN', 'YOUR_TOKEN')
# GameAPIを呼ぶ際のインターバル
GAME_INFO_SLEEP_TIME = 5000


def call_api(x: str) -> bytes:
    with urlopen(f'{GAME_SERVER}{x}') as res:
        return res.read()


def call_move(index: int, x: int, y: int) -> dict:
    res_str = call_api(f'/api/move/{TOKEN}/{index}-{x}-{y}')
    res_json = json.loads(res_str)
    return res_json


def call_move_next(index: int, x: int, y: int) -> dict:
    res_str = call_api(f'/api/move_next/{TOKEN}/{index}-{x}-{y}')
    res_json = json.loads(res_str)
    return res_json


def call_game() -> dict:
    res_str = call_api(f'/api/game/{TOKEN}')
    res_json = json.loads(res_str)
    return res_json


def call_master_data() -> dict:
    res_str = call_api(f'/api/master_data')
    res_json = json.loads(res_str)
    return res_json


class Bot:
    def __init__(self):
        self.master_data = call_master_data()
        self.game_info = call_game()
        self.start_game_time_ms = self.game_info['now']
        print("Start:", self.start_game_time_ms)
        self.start_time_ms = time.perf_counter_ns() // 1000000
        self.next_call_game_info_time_ms = self.get_now_game_time_ms() + GAME_INFO_SLEEP_TIME
        self.agent_move_finish_ms = [0] * self.master_data['num_agent']
        self.agent_move_point_queue = [[] for _ in range(self.master_data['num_agent'])]
        self.agent_last_point = [[] for _ in range(self.master_data['num_agent'])]
        for i in range(self.master_data['num_agent']):
            agent_move = self.game_info['agent'][i]['move']
            self.agent_last_point[i] = [agent_move[-1]['x'], agent_move[-1]['y']]
            self.set_move_point(i)

    # タスクを取得
    def choice_task(self) -> dict:
        tasks = self.game_info['task']
        return random.choice(tasks)

    def get_now_game_time_ms(self) -> int:
        now_ms = time.perf_counter_ns() // 1000000
        return self.start_game_time_ms + (now_ms - self.start_time_ms)

    def get_checkpoint(self, name: str) -> Tuple[int, int]:
        index = ord(name) - ord('A')
        checkpoint = self.master_data['checkpoints'][index]
        return checkpoint['x'], checkpoint['y']

    # 移動予定を設定
    def set_move_point(self, index: int):
        next_task = self.choice_task()
        print("Agent#{} next task:{}".format(index, next_task['s']))
        for i in range(len(next_task['s'])):
            before_x = self.agent_last_point[index][0]
            before_y = self.agent_last_point[index][1]
            move_x, move_y = self.get_checkpoint(next_task['s'][i])

            # 移動先が同じ場所の場合判定が入らないため別の箇所に移動してからにする
            if move_x == before_x and move_y == before_y:
                tmp_x = (before_x-1) if before_x > 10 else (before_x+1)#self.master_data['area_size'] // 2
                tmp_y = before_y#self.master_data['area_size'] // 2
                self.agent_move_point_queue[index].append([tmp_x, tmp_y])

            self.agent_move_point_queue[index].append([move_x, move_y])
            self.agent_last_point[index] = [move_x, move_y]

    def move_next(self, index: int) -> dict:
        move_next_point = self.agent_move_point_queue[index].pop(0)
        move_next_res = call_move_next(index+1, move_next_point[0], move_next_point[1])
        print("Agent#{} move_next to ({}, {})".format(index+1, move_next_point[0], move_next_point[1]))

        if move_next_res['status'] != 'ok':
            print(move_next_res)
            exit(1)

        assert len(move_next_res['move']) > 1

        self.agent_move_finish_ms[index] = move_next_res['move'][1]['t'] + 100

        # タスクを全てやりきったら次のタスクを取得
        if not self.agent_move_point_queue[index]:
            self.set_move_point(index)

        return move_next_res

    # game_infoの状態でのスコアを取得
    def get_now_score(self) -> float:
        score = 0.0
        tasks = self.game_info['task']
        for i in range(len(tasks)):
            if tasks[i]['total'] == 0:
                continue
            score += tasks[i]['weight'] * tasks[i]['count'] / tasks[i]['total']

        return score

    def solve(self):
        while True:
            now_game_time_ms = self.get_now_game_time_ms()

            # エージェントを移動させる
            for i in range(self.master_data['num_agent']):
                if self.agent_move_finish_ms[i] < now_game_time_ms:
                    move_next_res = self.move_next(i)
                    # 次の移動予定がない場合もう一度実行する
                    if len(move_next_res['move']) == 2:
                        self.move_next(i)

            # ゲーム情報更新
            if self.next_call_game_info_time_ms < now_game_time_ms:
                print('Update GameInfo')
                self.game_info = call_game()
                self.next_call_game_info_time_ms = self.get_now_game_time_ms() + GAME_INFO_SLEEP_TIME
                print("Score: {}".format(self.get_now_score()))

            time.sleep(0.5)


if __name__ == "__main__":
    bot = Bot()
    bot.solve()

Start: 1334354
Agent#0 next task:TTUEBBFN
Agent#1 next task:TTUEBBFN
Agent#2 next task:AND
Agent#3 next task:OGAAPLGJ
Agent#4 next task:QKEOMSUNVH
Agent#1 move_next to (30, 27)
Agent#1 move_next to (29, 27)
Agent#2 move_next to (30, 27)
Agent#2 move_next to (29, 27)
Agent#3 move_next to (12, 7)
Agent#3 move_next to (29, 13)
Agent#4 move_next to (12, 15)
Agent#4 move_next to (28, 18)
Agent#5 move_next to (15, 11)
Agent#5 move_next to (30, 6)
Agent#1 move_next to (30, 27)
Agent#1 move_next to (15, 19)
Agent#2 move_next to (30, 27)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (12, 7)
Agent#5 move_next to (25, 9)
Agent#3 move_next to (3, 20)
Agent#2 next task:GAME
Agent#4 move_next to (11, 7)
Agent#5 move_next to (12, 15)
Agent#1 move_next to (25, 9)
Agent#1 move_next to (5, 24)
Agent#2 move_next to (25, 9)
Agent#2 move_next to (5, 24)
Agent#5 move_next to (3, 0)
Agent#3 move_next to (28, 18)
Update GameInfo
Score: 51.27786543910068
Agent#4 move_next to (12, 7)
Agent#4 move_next to (

Agent#3 move_next to (30, 27)
Agent#1 move_next to (3, 0)
Agent#3 move_next to (29, 27)
Agent#4 move_next to (29, 13)
Agent#5 move_next to (30, 27)
Agent#2 move_next to (29, 27)
Agent#1 move_next to (25, 9)
Agent#0 next task:MUAOGAAPL
Agent#4 move_next to (3, 20)
Agent#3 next task:WBEFE
Update GameInfo
Score: 53.08606331609176
Agent#1 move_next to (3, 0)
Agent#5 move_next to (29, 27)
Agent#2 move_next to (30, 27)
Agent#2 move_next to (15, 19)
Agent#1 next task:TTUEBBFN
Agent#3 move_next to (30, 27)
Agent#3 move_next to (9, 29)
Agent#4 move_next to (11, 11)
Agent#1 move_next to (15, 19)
Agent#5 move_next to (30, 27)
Agent#5 move_next to (9, 29)
Agent#2 move_next to (30, 27)
Agent#2 move_next to (29, 27)
Agent#3 move_next to (25, 9)
Agent#2 next task:LUCK
Agent#3 move_next to (9, 29)
Agent#4 move_next to (5, 24)
Agent#1 move_next to (12, 7)
Agent#5 move_next to (25, 9)
Agent#4 next task:NAGQCBQN
Agent#5 move_next to (29, 13)
Update GameInfo
Score: 53.07346084665134
Agent#2 move_next to (

Agent#2 move_next to (17, 4)
Agent#3 move_next to (15, 11)
Agent#5 move_next to (12, 7)
Agent#1 move_next to (7, 5)
Agent#2 move_next to (5, 24)
Agent#4 move_next to (15, 11)
Agent#1 move_next to (15, 19)
Update GameInfo
Score: 53.42577480360623
Agent#2 move_next to (15, 11)
Agent#3 move_next to (29, 13)
Agent#2 next task:BATTLE
Agent#5 move_next to (17, 4)
Agent#4 move_next to (30, 6)
Agent#3 move_next to (5, 24)
Agent#5 move_next to (21, 23)
Agent#1 move_next to (29, 13)
Agent#0 next task:LUCK
Agent#2 move_next to (29, 13)
Agent#1 next task:UXACPMWUSK
Agent#4 move_next to (25, 9)
Agent#5 move_next to (3, 0)
Agent#3 move_next to (12, 7)
Agent#1 move_next to (9, 29)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (12, 15)
Agent#4 move_next to (3, 0)
Agent#5 move_next to (11, 11)
Agent#1 move_next to (15, 19)
Update GameInfo
Score: 53.80184500465331
Agent#2 move_next to (21, 0)
Agent#3 move_next to (30, 27)
Agent#4 move_next to (0, 28)
Agent#2 move_next to (12, 7)
Agent#5 move_next t

Agent#5 move_next to (11, 15)
Agent#3 move_next to (29, 27)
Agent#2 move_next to (21, 0)
Agent#1 move_next to (30, 27)
Agent#0 next task:KJQTTU
Agent#4 move_next to (11, 11)
Agent#5 move_next to (12, 15)
Agent#5 move_next to (3, 20)
Agent#4 next task:KTAILJ
Agent#2 move_next to (12, 7)
Agent#3 move_next to (30, 27)
Agent#3 move_next to (15, 19)
Agent#5 move_next to (30, 6)
Agent#5 move_next to (30, 27)
Update GameInfo
Score: 52.7706000298217
Agent#1 move_next to (30, 6)
Agent#2 move_next to (17, 4)
Agent#4 move_next to (15, 19)
Agent#3 move_next to (25, 9)
Agent#3 move_next to (5, 24)
Agent#2 move_next to (21, 23)
Agent#4 move_next to (0, 28)
Agent#2 move_next to (3, 0)
Agent#3 move_next to (6, 24)
Agent#4 move_next to (30, 6)
Agent#3 next task:BATTLE
Agent#5 move_next to (12, 7)
Agent#1 move_next to (29, 30)
Agent#2 move_next to (11, 11)
Agent#4 move_next to (5, 24)
Agent#5 move_next to (24, 17)
Update GameInfo
Score: 52.41476287622576
Agent#1 move_next to (15, 11)
Agent#3 move_next t

Update GameInfo
Score: 51.540268247798075
Agent#4 move_next to (11, 11)
Agent#2 move_next to (21, 0)
Agent#3 move_next to (15, 19)
Agent#5 move_next to (29, 13)
Agent#1 move_next to (25, 9)
Agent#0 next task:K
Agent#5 move_next to (30, 27)
Agent#2 move_next to (12, 7)
Agent#3 move_next to (21, 0)
Agent#4 move_next to (5, 24)
Agent#5 move_next to (25, 9)
Agent#1 move_next to (30, 6)
Agent#0 next task:EKVLDEJS
Agent#3 move_next to (12, 7)
Agent#2 move_next to (17, 4)
Agent#4 move_next to (25, 9)
Agent#3 next task:WBEFE
Agent#5 move_next to (0, 28)
Update GameInfo
Score: 51.329636937298844
Agent#1 move_next to (25, 9)
Agent#2 move_next to (21, 23)
Agent#3 move_next to (17, 4)
Agent#1 move_next to (30, 6)
Agent#2 move_next to (3, 0)
Agent#4 move_next to (11, 11)
Agent#1 move_next to (15, 24)
Agent#3 move_next to (21, 23)
Agent#5 move_next to (30, 27)
Agent#4 next task:UXACPMWUSK
Agent#1 move_next to (9, 29)
Agent#3 move_next to (3, 0)
Agent#2 move_next to (11, 11)
Agent#4 move_next to (5, 

...

Agent#3 move_next to (5, 24)
Agent#1 move_next to (25, 9)
Agent#2 move_next to (7, 5)
Agent#3 move_next to (25, 9)
Agent#4 move_next to (11, 11)
Agent#5 move_next to (28, 18)
Update GameInfo
Score: 66.4702171629903
Agent#3 move_next to (30, 27)
Agent#5 move_next to (15, 11)
Agent#1 move_next to (29, 30)
Agent#2 move_next to (25, 9)
Agent#1 next task:OGAAPLGJ
Agent#4 move_next to (7, 5)
Agent#5 move_next to (17, 4)
Agent#1 move_next to (8, 16)
Agent#0 next task:KQSWFU
Agent#2 move_next to (12, 15)
Agent#3 move_next to (24, 17)
Agent#4 move_next to (15, 19)
Agent#3 next task:PMWUSKDDS
Agent#4 move_next to (21, 23)
Agent#5 move_next to (5, 24)
Update GameInfo
Score: 66.37775337762184
Agent#2 move_next to (28, 18)
Agent#3 move_next to (9, 29)
Agent#5 move_next to (15, 11)
Agent#1 move_next to (30, 6)
Agent#4 move_next to (3, 0)
Agent#3 move_next to (5, 24)
Agent#2 move_next to (12, 7)
Agent#4 move_next to (11, 11)
Agent#5 move_next to (29, 13)
Agent#4 next task:KLAB
Agent#1 move_next to (1

Agent#2 move_next to (9, 29)
Agent#4 move_next to (24, 9)
Agent#1 move_next to (15, 24)
Agent#2 move_next to (7, 5)
Agent#1 next task:LGJKWPC
Agent#5 move_next to (15, 19)
Agent#4 next task:GOOD
Agent#1 move_next to (15, 11)
Agent#3 move_next to (15, 19)
Agent#4 move_next to (25, 9)
Agent#4 move_next to (15, 11)
Agent#5 move_next to (28, 18)
Agent#1 move_next to (9, 29)
Agent#2 move_next to (9, 29)
Agent#3 move_next to (0, 28)
Agent#4 move_next to (14, 11)
Agent#4 move_next to (15, 11)
Agent#5 move_next to (12, 15)
Update GameInfo
Score: 66.93501243047153
Agent#1 move_next to (7, 5)
Agent#0 next task:KQSWFU
Agent#3 move_next to (30, 6)
Agent#2 next task:UXACPMWUSK
Agent#4 move_next to (12, 7)
Agent#4 move_next to (15, 19)
Agent#3 next task:RXCFZTWBE
Agent#4 move_next to (8, 16)
Agent#5 move_next to (11, 15)
Agent#2 move_next to (28, 18)
Agent#3 move_next to (15, 19)
Agent#1 move_next to (30, 6)
Agent#4 move_next to (21, 0)
Agent#5 move_next to (12, 15)
Agent#5 move_next to (3, 20)
Agen

Agent#1 move_next to (1, 10)
Agent#0 next task:K
Agent#2 move_next to (25, 9)
Agent#5 move_next to (29, 13)
Agent#4 next task:OGAAPLGJ
Agent#3 move_next to (7, 5)
Agent#2 move_next to (30, 27)
Agent#4 move_next to (9, 29)
Agent#4 move_next to (28, 18)
Agent#5 move_next to (12, 15)
Agent#1 move_next to (30, 6)
Agent#0 next task:KLAB
Agent#3 move_next to (15, 19)
Agent#3 move_next to (15, 24)
Agent#2 next task:BATTLE
Agent#4 move_next to (29, 30)
Agent#5 move_next to (28, 18)
Update GameInfo
Score: 67.28010606573191
Agent#2 move_next to (24, 17)
Agent#3 move_next to (5, 24)
Agent#5 move_next to (12, 7)
Agent#1 move_next to (29, 6)
Agent#2 move_next to (9, 29)
Agent#3 move_next to (12, 7)
Agent#4 move_next to (30, 6)
Agent#3 move_next to (30, 27)
Agent#4 move_next to (11, 11)
Agent#5 move_next to (11, 7)
Agent#2 move_next to (5, 24)
Agent#1 move_next to (30, 6)
Agent#1 move_next to (9, 29)
Agent#2 move_next to (7, 5)
Agent#1 next task:BATTLE
Agent#3 move_next to (29, 27)
Agent#4 move_next

Agent#2 move_next to (17, 4)
Agent#5 move_next to (20, 23)
Agent#1 move_next to (5, 24)
Agent#3 move_next to (5, 24)
Agent#2 move_next to (5, 24)
Agent#4 move_next to (11, 11)
Agent#3 move_next to (25, 9)
Agent#5 move_next to (21, 23)
Agent#5 move_next to (8, 16)
Agent#1 move_next to (7, 5)
Agent#0 next task:LGJKWPC
Agent#2 move_next to (15, 11)
Agent#4 move_next to (28, 18)
Update GameInfo
Score: 67.72980720739659
Agent#1 move_next to (9, 29)
Agent#4 move_next to (30, 6)
Agent#3 move_next to (30, 27)
Agent#5 move_next to (9, 29)
Agent#5 move_next to (1, 10)
Agent#2 move_next to (29, 13)
Agent#1 next task:YHTDDUVA
Agent#4 move_next to (15, 11)
Agent#1 move_next to (28, 18)
Agent#5 move_next to (12, 7)
Agent#2 move_next to (0, 14)
Agent#3 move_next to (24, 17)
Agent#4 move_next to (0, 28)
Agent#3 next task:BATTLE
Agent#1 move_next to (29, 30)
Agent#2 move_next to (1, 10)
Agent#5 move_next to (10, 20)
Agent#3 move_next to (9, 29)
Agent#4 move_next to (5, 24)
Update GameInfo
Score: 3401.0

KeyboardInterrupt: 